In [1]:
# INSTALLS
!pip install lightkurve
!pip install tslearn 
!pip install pytictoc

# IMPORTING NECESSARY LIBRARIES
import lightkurve as lk
import pandas as pd
import numpy as np
from tslearn.utils import to_time_series_dataset
from tslearn.preprocessing import TimeSeriesResampler
from pytictoc import TicToc

# IMPORT AND PROCESSING THE KEPLER ID DATA
finalData = pd.read_csv('finalData.csv')
finalData = finalData[finalData['koi_disposition'].str.contains('CANDIDATE')==False] # removing 'candidate' rows
finalData = finalData.drop_duplicates(subset=['kepid']) # removing duplicates where kepid is repeated ==> CONTROVERSIAL!!! to deal with multiple planets (0, 1, 1+) classification, need raw data
finalData = finalData.replace(['CONFIRMED', 'FALSE POSITIVE'], [1, 0]) # converting 'confirmed' / 'false positive' into 1 / 0
data = finalData
data = data.reset_index(drop=True)

initial_X1 = []
initial_Y1 = []
i = 0 # dummy variable to verify the loop is working

for kepid in data['kepid'].iloc[2001:3000]:
  t = TicToc()
  start_time = t.tic()


  try: 
    # DOWNLOADING THE DATA
    KIC = 'KIC ' + str(kepid)
    lcs = lk.search_lightcurve(KIC, author='kepler', cadence='long').download_all()

    # FINDING THE PERIOD, T0 AND DURATION FOR PROCESSING
    row_number = finalData[finalData['kepid'] == kepid].index[0]
    period, t0, duration_hours =  finalData['koi_period'][row_number],  finalData['koi_time0bk'][row_number],  finalData['koi_duration'][row_number]
    
    # PROCESSING THE LIGHTKURVE DATA
    lc_raw = lcs.stitch()
    lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4)
    temp_fold = lc_clean.fold(period, epoch_time=t0)
    fractional_duration = (duration_hours / 24.0) / period
    phase_mask = np.abs(temp_fold.phase.value) < (fractional_duration * 1.5)
    transit_mask = np.in1d(lc_clean.time.value, temp_fold.time_original.value[phase_mask])
    lc_flat, trend_lc = lc_clean.flatten(return_trend=True, mask=transit_mask)
    lc_fold = lc_flat.fold(period, epoch_time=t0)
    
    # CREATING THE GLOBAL VIEW
    lc_global = lc_fold.bin(time_bin_size=0.005).normalize() - 1
    lc_global = (lc_global / np.abs(lc_global.flux.min()) ) * 2.0 + 1


    # CONVERTING TO PANDAS DF
    lc_global = lc_global.to_pandas()

    # CREATE LISTS OF X (TIMESERIES DATA) AND Y (LABEL)
    initial_X1.append(lc_global['flux'].tolist())
    initial_Y1.append(finalData['koi_disposition'][row_number])

    print(i)
    i += 1

    end_time = t.toc()
    print(end_time)


  except: 
    pass


# converting initial_X and initial_Y to pandas df
df1 = pd.DataFrame(initial_X1)
df1['target'] = initial_Y1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 247 kB 29.9 MB/s 
     |████████████████████████████████| 98 kB 9.8 MB/s 
     |████████████████████████████████| 41 kB 229 kB/s 
     |████████████████████████████████| 4.5 MB 48.4 MB/s 
     |████████████████████████████████| 832 kB 72.6 MB/s 
     |████████████████████████████████| 48 kB 6.1 MB/s 
     |████████████████████████████████| 4.1 MB 42.6 MB/s 
  Created wheel for fbpca: filename=fbpca-1.0-py3-none-any.whl size=11375 sha256=75c12f3f3f99169e0478e905fb7552924fa60209d11601bfa30f8992b2b85fa3
  Stored in directory: /root/.cache/pip/wheels/93/08/0c/1b9866c35c8d3f136d100dfe88036a32e0795437daca089f70
  Created wheel for memoization: filename=memoization-0.4.0-py3-none-any.whl size=50467 sha256=8167551a0d92212c4faf830f9e715f48292cbd17bc0480066e751914216c9bd3
  Stored in directory: /root/.cache/pip/wheels/38/f7/65/161985e7311dd484a23b3a5c9149995d

<string>:6: RuntimeWarning: Mean of empty slice
<string>:6: RuntimeWarning: Degrees of freedom <= 0 for slice.


53
Elapsed time is 8.323392 seconds.
None
54
Elapsed time is 10.916262 seconds.
None
55
Elapsed time is 7.066841 seconds.
None
56
Elapsed time is 7.872130 seconds.
None
57
Elapsed time is 7.835445 seconds.
None
58
Elapsed time is 7.226161 seconds.
None
59
Elapsed time is 11.738976 seconds.
None
60
Elapsed time is 9.547040 seconds.
None
61
Elapsed time is 7.412372 seconds.
None
62
Elapsed time is 6.072122 seconds.
None
63
Elapsed time is 63.322042 seconds.
None
64
Elapsed time is 8.029764 seconds.
None
65
Elapsed time is 6.869355 seconds.
None
66
Elapsed time is 15.371837 seconds.
None
67
Elapsed time is 11.861995 seconds.
None
68
Elapsed time is 8.371820 seconds.
None
69
Elapsed time is 4.686336 seconds.
None
70
Elapsed time is 4.877748 seconds.
None
71
Elapsed time is 19.258189 seconds.
None
72
Elapsed time is 13.607098 seconds.
None
73
Elapsed time is 7.858390 seconds.
None
74
Elapsed time is 8.112888 seconds.
None
75
Elapsed time is 7.159651 seconds.
None
76
Elapsed time is 10.27625

<string>:6: RuntimeWarning: Mean of empty slice
<string>:6: RuntimeWarning: Degrees of freedom <= 0 for slice.


120
Elapsed time is 8.125745 seconds.
None
121
Elapsed time is 12.070638 seconds.
None
122
Elapsed time is 5.000886 seconds.
None
123
Elapsed time is 14.614406 seconds.
None
124
Elapsed time is 8.662314 seconds.
None
125
Elapsed time is 8.270350 seconds.
None
126
Elapsed time is 8.252585 seconds.
None
127
Elapsed time is 14.401809 seconds.
None
128
Elapsed time is 4.894755 seconds.
None
129
Elapsed time is 7.751950 seconds.
None
130
Elapsed time is 10.519930 seconds.
None
131
Elapsed time is 9.493419 seconds.
None
132
Elapsed time is 9.201842 seconds.
None
133
Elapsed time is 8.903509 seconds.
None
134
Elapsed time is 12.057262 seconds.
None
135
Elapsed time is 8.525037 seconds.
None
136
Elapsed time is 7.765995 seconds.
None
137
Elapsed time is 14.458615 seconds.
None
138
Elapsed time is 12.709553 seconds.
None
139
Elapsed time is 10.920691 seconds.
None
140
Elapsed time is 9.921985 seconds.
None
141
Elapsed time is 9.016993 seconds.
None
142
Elapsed time is 8.051291 seconds.
None
143

...

904
Elapsed time is 14.419116 seconds.
None
905
Elapsed time is 8.973368 seconds.
None


<string>:6: RuntimeWarning: Mean of empty slice
<string>:6: RuntimeWarning: Degrees of freedom <= 0 for slice.


906
Elapsed time is 18.098036 seconds.
None
907
Elapsed time is 9.081244 seconds.
None
908
Elapsed time is 9.077425 seconds.
None
909
Elapsed time is 9.477921 seconds.
None
910
Elapsed time is 54.490524 seconds.
None
911
Elapsed time is 45.343465 seconds.
None
912
Elapsed time is 9.116121 seconds.
None
913
Elapsed time is 9.142350 seconds.
None
914
Elapsed time is 5.646964 seconds.
None
915
Elapsed time is 1.518608 seconds.
None
916
Elapsed time is 8.516919 seconds.
None
917
Elapsed time is 7.542397 seconds.
None
918
Elapsed time is 7.671308 seconds.
None
919
Elapsed time is 10.353078 seconds.
None
920
Elapsed time is 78.937580 seconds.
None
921
Elapsed time is 39.050983 seconds.
None
922
Elapsed time is 39.045525 seconds.
None
923
Elapsed time is 8.579420 seconds.
None
924
Elapsed time is 64.653072 seconds.
None
925
Elapsed time is 82.793624 seconds.
None
926
Elapsed time is 8.383091 seconds.
None
927
Elapsed time is 7.211957 seconds.
None
928
Elapsed time is 8.811739 seconds.
None
92

<string>:6: RuntimeWarning: Mean of empty slice
<string>:6: RuntimeWarning: Degrees of freedom <= 0 for slice.


964
Elapsed time is 93.508959 seconds.
None
965
Elapsed time is 19.505554 seconds.
None
966
Elapsed time is 75.867345 seconds.
None
967
Elapsed time is 9.167743 seconds.
None
968
Elapsed time is 95.156398 seconds.
None
969
Elapsed time is 7.733159 seconds.
None
970
Elapsed time is 8.190191 seconds.
None
971
Elapsed time is 7.621834 seconds.
None
972
Elapsed time is 8.494982 seconds.
None
973
Elapsed time is 97.074373 seconds.
None
974
Elapsed time is 8.210126 seconds.
None
975
Elapsed time is 14.507113 seconds.
None


In [2]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
path = '/content/drive/My Drive/GSOC/WEEK 7/FINALoutput2001_3000.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  df1.to_csv(f)

Mounted at /content/drive


In [3]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

